# Using a D-Wave quantum computer to determine commutativity of finite-dimensional algebras

In this notebook, we implement the methods described in <a href="https://www.researchgate.net/publication/330605396_Experiments_Testing_the_Commutativity_of_Finite-Dimensional_Algebras_with_a_Quantum_Adiabatic_Algorithm_Testing_Commutativity_FD_Algebras_with_a_QAA">Experiments Testing the Commutativity of Finite-Dimensional Algebras with a Quantum Adiabatic Algorithm (Combarro, Ranilla, Rúa 2019)</a> to test the commutativity of finite-dimensional algebras.

We provide code for the general case and then solve a simple example with both exact and heuristic classical methods and with the use of a D-Wave quantum annealer.



First, we import all the need libraries and define a function that gives the constants matriz of random algebra with a fixed number of non-commuting pairs

In [1]:
import numpy as np
from random import randint

def randomAlgebra(m,c):
# Returns a random algebra of size 2^m with c non-commuting pairs
  n=2**m
  M=np.zeros((n,n,n))
  f=0
  while f<c:
    #Bucar una posicion para poner un fallo
    i=randint(0,n-1)
    j=i
    while j==i:
      j=randint(0,n-1)
    k=randint(0,n-1)
    if M[i,j,k]==M[j,i,k]:
      M[i,j,k]=1
      M[j,i,k]=0
      f=f+1
  return M

This how the matrix for a simple case would look. Notice that there is exactly one case in which $M_{ijk}$ is different from $M_{jik}$

In [2]:
M=randomAlgebra(1,1)
print(M)

[[[0. 0.]
  [0. 1.]]

 [[0. 0.]
  [0. 0.]]]


Now, we define a function that, given (the constants matrix of) and algebra, computes the oracle that will mark those pairs of elements that do not commute.

In [3]:
def oracle(M):
    n = M.shape[0]
    O = np.zeros((n,n,n))
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if M[i,j,k]!=M[j,i,k]:
                    O[i,j,k]=1
    return O

This is the oracle for algebra $M$

In [4]:
O=oracle(M)
print(O)

[[[0. 0.]
  [0. 1.]]

 [[0. 1.]
  [0. 0.]]]


We construct a Ising model in order to find the non-commuting pairs of the matrix

In [5]:
def Ising(O):
    J ={}
    n = O.shape[0]
    for i in range(n):
        for j in range(i):
            for k in range(n):
                if O[i,j,k]==1:
                    J[(i,j)]=1
                    break
    return J

These are the coefficients for our case

In [6]:
J=Ising(O)
print(J)

{(1, 0): 1}


We instantiate the model. Notice that the coefficients for individual spins (the $h$) are all 0

In [7]:
import dimod

h = {}
model = dimod.BinaryQuadraticModel(h, J, 0.0, dimod.SPIN)

print(model)

BinaryQuadraticModel({1: 0, 0: 0}, {(1, 0): 1}, 0.0, Vartype.SPIN)


We can solve this model exactly

In [8]:
from dimod.reference.samplers import ExactSolver
sampler = ExactSolver()
solution = sampler.sample(model)
print(solution)

    0   1  energy  num_occ.
0  +1  -1    -1.0         1
1  -1  +1    -1.0         1
2  -1  -1     1.0         1
3  +1  +1     1.0         1

[ 4 rows, 2 variables ]


Or with simulated annealing

In [9]:
sampler = dimod.SimulatedAnnealingSampler()
response = sampler.sample(model, num_reads=10)
print(response)

    1   0  energy  num_occ.
0  -1  +1    -1.0         1
1  +1  -1    -1.0         1
2  -1  +1    -1.0         1
3  +1  -1    -1.0         1
4  -1  +1    -1.0         1
5  +1  -1    -1.0         1
6  +1  -1    -1.0         1
7  -1  +1    -1.0         1
8  -1  +1    -1.0         1
9  +1  -1    -1.0         1

[ 10 rows, 2 variables ]


And, of course, also using D-Wave's quantum annealer 

In [10]:
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
sampler = EmbeddingComposite(DWaveSampler())
response = sampler.sample(model, num_reads=5000)
print(response)

    1   0  energy  num_occ.  chain_b.
0  +1  -1    -1.0      2329       0.0
1  -1  +1    -1.0      2664       0.0
2  -1  -1     1.0         4       0.0
3  +1  +1     1.0         3       0.0

[ 4 rows, 2 variables ]
